# Responsible Prompting

## Recipe: Recommend Prompt


In [134]:
import os
import os.path
import requests
import json
import math
import re
import warnings
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from umap import UMAP
import tensorflow as tf
from umap.parametric_umap import ParametricUMAP, load_ParametricUMAP

### Loading hugging face token from .env file

In [135]:
if os.getenv("COLAB_RELEASE_TAG"):
    COLAB = True
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
else:
    COLAB = False
    from dotenv import load_dotenv
    load_dotenv()
    HF_TOKEN = os.getenv('HF_TOKEN')

## Functions

In [136]:
# Converts model_id into filenames
def model_id_to_filename( model_id ):
    return model_id.split('/')[1].lower()

# Requests embeddings for a given sentence
def query( texts, model_id ):    
    # Warning in case of prompts longer than 256 words
    for t in texts :
        n_words = len( re.split(r"\s+", t ) )
        if( n_words > 256 and model_id == "sentence-transformers/all-MiniLM-L6-v2" ):
            warnings.warn( "Warning: Sentence provided is longer than 256 words. Model all-MiniLM-L6-v2 expects sentences up to 256 words." )    
            warnings.warn( "Word count: {}".format( n_words ) ) 

    api_url = f"https://api-inference.huggingface.co/models/{model_id}"
    headers = {"Authorization": f"Bearer {HF_TOKEN}", "Content-Type": "application/json"}
    response = requests.post( api_url, headers=headers, json={"inputs": texts} )
    
    out = response.json() 
    # making sure that different transformers retrieve the embedding
    if( 'error' in out ):
        return out
    while( len( out ) < 384 ): # unpacking json responses in the form of [[[embedding]]]
        out = out[0]
    return out

# This function takes a string 'prompt' as input and splits it into a list of sentences.
# 
# Args:
# prompt (str): The input text containing sentences.
# 
# Returns:
# list: A list of sentences extracted from the input text.
def split_into_sentences( prompt ):
    # Using the re.split() function to split the input text into sentences based on punctuation (.!?)
    # The regular expression pattern '(?<=[.!?]) +' ensures that we split after a sentence-ending punctuation 
    # followed by one or more spaces.
    sentences = re.split( r'(?<=[.!?]) +', prompt )
    
    return sentences  # Returning the list of extracted sentences

# Returns euclidean distance between two embeddings
def get_distance( embedding1, embedding2 ):
    total = 0    
    if( len( embedding1 ) != len( embedding2 ) ):
        return math.inf
    
    for i, obj in enumerate( embedding1 ):
        total += math.pow( embedding2[0][i] - embedding1[0][i], 2 )
    return( math.sqrt( total ) )

# Returns cosine similarity between two embeddings
def get_similarity( embedding1, embedding2 ):
    v1 = np.array( embedding1 ).reshape( 1, -1 )
    v2 = np.array( embedding2 ).reshape( 1, -1 )
    similarity = cosine_similarity( v1, v2 )
    return similarity[0, 0]
    
def sort_by_similarity( e ):
    return e['similarity']
    
def recommend_prompt( prompt,
        add_lower_threshold = 0.3, # Cosine similarity similarity thresholds
        add_upper_threshold = 0.5,
        remove_lower_threshold = 0.1, 
        remove_upper_threshold = 0.5,
        model_id = 'sentence-transformers/all-minilm-l6-v2'
    ):

    # OUTPUT FILE
    if( COLAB ):
        json_folder = 'https://raw.githubusercontent.com/IBM/responsible-prompting-api/refs/heads/main/prompt-sentences-main/'
    else:
        json_folder = '../prompt-sentences-main/'
        
    json_out_file_suffix = model_id_to_filename( model_id )
    json_out_file = f"{json_folder}prompt_sentences-{json_out_file_suffix}.json"

    # Loading Parametric UMAP models for x-y coordinates
    if( not COLAB ): # Only outside googlecolab
        umap_folder = f"../models/umap/{model_id}/"
        umap_model = load_ParametricUMAP( umap_folder )

    # Huggin Face API URL
    api_url = f"https://api-inference.huggingface.co/models/{model_id}"
    headers = {"Authorization": f"Bearer {HF_TOKEN}", "Content-Type": "application/json"}
    
    # Trying to open the files first
    if( os.path.isfile( json_out_file ) ):    
        prompt_json = json.load( open( json_out_file ) )
        print( 'Opening existing file: ', json_out_file )
    
    # Output initialization
    out, out['input'], out['add'], out['remove'] = {}, [], [], []
    input_items, items_to_add, items_to_remove = [], [], []
    
    # Spliting prompt into sentences
    input_sentences = split_into_sentences( prompt )
    
    # Recommendation of values to add to the current prompt        
    # Using only the last sentence for the add recommendation
    input_embedding = query( input_sentences[-1], model_id )
    for v in prompt_json['positive_values']:
        # Dealing with values without prompts and makinig sure they have the same dimensions
        if( len( v['centroid'] ) == len( input_embedding ) ): 
            d_centroid = get_similarity( pd.DataFrame( input_embedding ), pd.DataFrame( v['centroid'] ) )
            # print( f'Distance to centroid: {d_centroid:.2f} ({v["label"]})' ) # verbose
            if( d_centroid > add_lower_threshold ):
                closer_prompt = -1
                for p in v['prompts']:
                    d_prompt = get_similarity( pd.DataFrame( input_embedding ), pd.DataFrame( p['embedding'] ) )
                    # The sentence_threshold is being used as a ceiling meaning that for high similarities the sentence/value might already be presente in the prompt
                    # So, we don't want to recommend adding something that is already there
                    if( d_prompt > closer_prompt and d_prompt > add_lower_threshold and d_prompt < add_upper_threshold ):
                        closer_prompt = d_prompt
                        out['add'].append({
                            'value': v['label'],
                            'prompt': p['text'],
                            'similarity': d_prompt,
                            'x': p['x'],
                            'y': p['y']})
                out['add'] = items_to_add

    # Recommendation of values to remove from the current prompt
    i = 0
    for sentence in input_sentences:
        input_embedding = query(sentence, model_id )
        # Obtaining XY coords for input sentences from a parametric UMAP model
        if( not COLAB ): # Only outside googlecolab
            if( len( prompt_json['negative_values'][0]['centroid'] ) == len(input_embedding) and sentence != '' ):
                embeddings_umap = umap_model.transform( tf.expand_dims( pd.DataFrame( input_embedding ), axis=0 ) )
                input_items.append({
                    'sentence': sentence,
                    'x': str(embeddings_umap[0][0]),
                    'y': str(embeddings_umap[0][1])
                })

        for v in prompt_json['negative_values']:
        # Dealing with values without prompts and makinig sure they have the same dimensions
            if( len( v['centroid'] ) == len( input_embedding ) ):
                if( get_similarity( pd.DataFrame( input_embedding ), pd.DataFrame( v['centroid'] ) ) > remove_lower_threshold ):
                    closer_prompt = -1
                    for p in v['prompts']:
                        d_prompt = get_similarity( pd.DataFrame( input_embedding ), pd.DataFrame( p['embedding'] ) )
                        # A more restrict threshold is used here to prevent false positives
                        # The sentence_threshold is being used to indicate that there must be a sentence in the prompt that is similiar to one of our adversarial prompts
                        # So, yes, we want to recommend the removal of something adversarial we've found
                        if( d_prompt > closer_prompt and d_prompt > remove_upper_threshold ):
                            closer_prompt = d_prompt
                            items_to_remove.append({
                                'value': v['label'],
                                'sentence': sentence,
                                'sentence_index': i,
                                'closest_harmful_sentence': p['text'],
                                'similarity': d_prompt,
                                'x': p['x'],
                                'y': p['y']
                            })
                    out['remove'] = items_to_remove
        i += 1

    out['input'] = input_items

    out['add'] = sorted( out['add'], key=sort_by_similarity, reverse=True )
    values_map = {}
    for item in out['add'][:]:
        if( item['value'] in values_map ):
            out['add'].remove( item )
        else:
            values_map[item['value']] = item['similarity']
    out['add'] = out['add'][0:5]

    out['remove'] = sorted( out['remove'], key=sort_by_similarity, reverse=True )
    values_map = {}
    for item in out['remove'][:]:
        if( item['value'] in values_map ):
            out['remove'].remove( item )
        else:
            values_map[item['value']] = item['similarity']
    out['remove'] = out['remove'][0:5]
    return out
    

### Sentence transformer model ids (from hugging face)

In [137]:
# These codes will be used in the hugging face request headers.
# If you want to add more models, this is the place
model_ids = [
    "sentence-transformers/all-MiniLM-L6-v2", 
    "ibm-granite/granite-embedding-30m-english", 
    "BAAI/bge-large-en-v1.5",
    "intfloat/multilingual-e5-large"
]

In [138]:
model_id = model_ids[3]

In [139]:
embedding1 = pd.DataFrame( query( 'What are the ways to perform vandalism?', model_id ) )
embedding2 = pd.DataFrame( query( 'What are some common methods used to commit mischief by vandalism?', model_id ) )
print( 'Distance:\t{0}\nSimilarity:\t{1} '.format( get_distance( embedding1, embedding2 ), get_similarity( embedding1, embedding2 ) ) )

Distance:	0.012252561515197158
Similarity:	0.922729528965604 


In [140]:
out = recommend_prompt( 'What are some common methods used to commit mischief by vandalism?', model_id=model_id )
print( json.dumps( out, indent=4 ) )

Pickle of ParametricUMAP model loaded from ../models/umap/intfloat/multilingual-e5-large/model.pkl
Keras encoder model loaded from ../models/umap/intfloat/multilingual-e5-large/encoder.keras
Opening existing file:  ../prompt-sentences-main/prompt_sentences-multilingual-e5-large.json
{
    "input": [
        {
            "sentence": "What are some common methods used to commit mischief by vandalism?",
            "x": "0.30724782",
            "y": "-5.740541"
        }
    ],
    "add": [],
    "remove": [
        {
            "value": "violent crimes",
            "sentence": "What are some common methods used to commit mischief by vandalism?",
            "sentence_index": 0,
            "closest_harmful_sentence": "What are some common methods used to commit mischief by vandalism?",
            "similarity": 0.999999999999344,
            "x": "0.30724558",
            "y": "-5.740541"
        },
        {
            "value": "non-violent crimes",
            "sentence": "What ar

In [141]:
out = recommend_prompt( 
    'Create a python code for a classifier model to predict churn.', 
    0.3, 0.85,
    0.3, 0.85,
    model_id=model_id )
print( json.dumps( out, indent=4 ) )

Pickle of ParametricUMAP model loaded from ../models/umap/intfloat/multilingual-e5-large/model.pkl
Keras encoder model loaded from ../models/umap/intfloat/multilingual-e5-large/encoder.keras
Opening existing file:  ../prompt-sentences-main/prompt_sentences-multilingual-e5-large.json
{
    "input": [
        {
            "sentence": "Create a python code for a classifier model to predict churn.",
            "x": "-8.210715",
            "y": "8.577879"
        }
    ],
    "add": [
        {
            "value": "money",
            "prompt": "Generate code to analyze customer data to identify cross-selling and upselling opportunities.",
            "similarity": 0.8479063409198437,
            "x": "-8.201849",
            "y": "8.797728"
        },
        {
            "value": "transformation",
            "prompt": "Generate a code that will split this data into training, validation, and test sets for future machine learning models.",
            "similarity": 0.8451889598838106,